## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [3]:
spark

In [4]:
sqlContext

### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [3]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [5]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [11]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
print(dbs.head(10))
dbs.toPandas()

[Row(databaseName='country_club'), Row(databaseName='default')]


,databaseName
0,country_club
1,default


# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [12]:
# File location and type
file_location_bookings = "./Bookings.csv"
file_location_facilities = "./Facilities.csv"
file_location_members = "./Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [18]:
members_df.toPandas()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,NaN,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,NaN,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,NaN,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,NaN,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1.0,2012-07-03 10:25:05
5,5,Butters,Gerald,"1065 Huntingdon Avenue, Boston",56754,(844) 078-4130,1.0,2012-07-09 10:44:09
6,6,Tracy,Burton,"3 Tunisia Drive, Boston",45678,(822) 354-9973,NaN,2012-07-15 08:52:55
7,7,Dare,Nancy,"6 Hunting Lodge Way, Boston",10383,(833) 776-4001,4.0,2012-07-25 08:59:12
8,8,Boothe,Tim,"3 Bloomsbury Close, Reading, 00234",234,(811) 433-2547,3.0,2012-07-25 16:02:35
9,9,Stibbons,Ponder,"5 Dragons Way, Winchester",87630,(833) 160-3900,6.0,2012-07-25 17:09:05


In [19]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [20]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [22]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

,database,tableName,isTemporary
0,country_club,bookings,False
1,country_club,facilities,False
2,country_club,members,False


# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [24]:
result = run_sql('''
                    select * 
                    FROM bookings 
                    LIMIT 3
                 ''')
result.toPandas()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2


# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [36]:
result = run_sql('''
                   select name from 
                   Facilities where membercost=0
                 ''')
result.toPandas()

,name
0,Badminton Court
1,Table Tennis
2,Snooker Table
3,Pool Table


####  Q2: How many facilities do not charge a fee to members?

In [37]:
result = run_sql('''
                   select count(name) from facilities where membercost=0
                 ''')
result.toPandas()

,count(name)
0,4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [41]:
result = run_sql('''
                   select facid,name,membercost,monthlymaintenance from Facilities where 
                   membercost< 0.20*monthlymaintenance
                ''')
result.toPandas()

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,2,Badminton Court,0.0,50
3,3,Table Tennis,0.0,10
4,4,Massage Room 1,9.9,3000
5,5,Massage Room 2,9.9,3000
6,6,Squash Court,3.5,80
7,7,Snooker Table,0.0,15
8,8,Pool Table,0.0,15


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [42]:
result = run_sql('''
                   select * from facilities where facid in (1,5) 
                ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [53]:
result = run_sql('''
                   select name, monthlymaintenance, 
                   CASE WHEN monthlymaintenance >100 then 'expensive' else 'cheap' END AS Type
                   from facilities
                 ''')
result.toPandas()

,name,monthlymaintenance,Type
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [55]:
result = run_sql('''
                    Select firstname, surname from members where joindate in (select max(joindate) from members)
                    
                 ''')
result.toPandas()

,firstname,surname
0,Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [72]:
result = run_sql('''
                   SELECT distinct name, member_name
                   from (select  name, surname||firstname member_name
                   from facilities,members, bookings
                   where bookings.memid = members.memid
                   and bookings.facid   = facilities.facid and name like 'Tennis Court%' 
                   ) order by member_name
                 ''')
result.toPandas()

,name,member_name
0,Tennis Court 1,BaderFlorence
1,Tennis Court 2,BaderFlorence
2,Tennis Court 2,BakerAnne
3,Tennis Court 1,BakerAnne
4,Tennis Court 1,BakerTimothy
5,Tennis Court 2,BakerTimothy
6,Tennis Court 1,BootheTim
7,Tennis Court 2,BootheTim
8,Tennis Court 2,ButtersGerald
9,Tennis Court 1,ButtersGerald


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [131]:
result = run_sql('''
                   select name, 
                   (case when members.memid = 0 
                   then members.surname 
                   else surname||firstname end) as member_name, 
                   (case when members.memid = 0 then bookings.slots * facilities.guestcost
                         else bookings.slots * facilities.membercost
                         end) as cost 
                   from bookings,  members, facilities 
                   where  bookings.memid =members.memid
                   and bookings.facid=facilities.facid
                   and date_trunc('day', starttime) = '2012-09-04'
                   and ((bookings.memid <> 0 and bookings.slots * facilities.membercost > 30)
                    OR (bookings.memid = 0 and bookings.slots * facilities.guestcost > 30))
                    order by cost desc
                 ''') 
result.toPandas()

,name,member_name,cost
0,Massage Room 1,GUEST,160.0
1,Massage Room 1,GUEST,160.0
2,Massage Room 1,GUEST,160.0
3,Massage Room 1,GUEST,160.0
4,Massage Room 1,GUEST,160.0
5,Massage Room 2,GUEST,160.0
6,Massage Room 2,GUEST,160.0
7,Tennis Court 1,GUEST,75.0
8,Tennis Court 1,GUEST,75.0
9,Tennis Court 1,GUEST,75.0


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [123]:
result = run_sql('''
                   select name, member_name, cost
                   from 
                   (
                   select name, 
                   (case when members.memid = 0 
                   then members.surname 
                   else surname||firstname end) as member_name, 
                     (case when members.memid= 0 then (slots *guestcost) 
                            else (slots * membercost) end) as cost
                    from bookings, facilities, members
                    where  facilities.facid= bookings.facid
                    and members.memid = bookings.memid
                    and date_trunc('day', starttime) = '2012-09-04'
                    )
                    where cost > 30
                    order by cost desc
                 ''')
result.toPandas()

,name,member_name,cost
0,Massage Room 1,GUEST,160.0
1,Massage Room 1,GUEST,160.0
2,Massage Room 1,GUEST,160.0
3,Massage Room 1,GUEST,160.0
4,Massage Room 1,GUEST,160.0
5,Massage Room 2,GUEST,160.0
6,Massage Room 2,GUEST,160.0
7,Tennis Court 1,GUEST,75.0
8,Tennis Court 1,GUEST,75.0
9,Tennis Court 1,GUEST,75.0


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [130]:
result = run_sql('''
                   
                   select name, sum(cost) revenue
                   from 
                   (select
                   name, 
                     (case when members.memid= 0 then (slots *guestcost) 
                            else (slots * membercost) end) as cost
                   from bookings, facilities, members
                    where  facilities.facid= bookings.facid
                    and members.memid = bookings.memid
                    )
                    group by name
                    having sum(cost) <1000
                    order by revenue
                 ''')
result.toPandas()

,name,revenue
0,Table Tennis,180.0
1,Snooker Table,240.0
2,Pool Table,270.0
